In [1]:
!pip install git+https://github.com/Priesemann-Group/covid19_inference_forecast.git

  Cloning https://github.com/Priesemann-Group/covid19_inference_forecast.git to /private/var/folders/qr/hdfcfrpx0hj50l5g7lwg7ygc0000gn/T/pip-req-build-xr6i3ejy
  Running command git clone -q https://github.com/Priesemann-Group/covid19_inference_forecast.git /private/var/folders/qr/hdfcfrpx0hj50l5g7lwg7ygc0000gn/T/pip-req-build-xr6i3ejy
  on a case-insensitive filesystem) and only one from the same
  colliding group is in the working tree:

    'DISCLAIMER.md'
    'disclaimer.md'
  Created wheel for covid19-inference: filename=covid19_inference-0.0.10-py3-none-any.whl size=27625 sha256=c8aa3f0400f19aac1e5b28821b74b25d9a688caa46b4980e4684b19941dda564
  Stored in directory: /private/var/folders/qr/hdfcfrpx0hj50l5g7lwg7ygc0000gn/T/pip-ephem-wheel-cache-7lt040l0/wheels/cc/33/51/375e62e25122e2286fa440949767c3f7f3e2e75f7a2cd4ca7e
Successfully built covid19-inference


In [2]:
import datetime
import time as time_module
import sys
import os 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import theano
import matplotlib
import pymc3 as pm

try: 
    import covid19_inference as cov19
except ModuleNotFoundError:
    sys.path.append('../..')
    import covid19_inference as cov19

path_to_save = '../../figures/'

INFO: This code is not longer developed, use our new module instead: github.com/Priesemann-Group/covid19_inference


In [ ]:

confirmed_cases = cov19.get_jhu_confirmed_cases()

country = 'US'
date_data_begin = datetime.datetime(2020,3,8)
date_data_end   = cov19.get_last_date(confirmed_cases)

#date_data_end   = datetime.datetime(2020,3,28)
num_days_data = (date_data_end-date_data_begin).days
diff_data_sim = 16 # should be significantly larger than the expected delay, in 
                   # order to always fit the same number of data points.
num_days_future = 180
date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim)
date_end_sim   = date_data_end   + datetime.timedelta(days = num_days_future)
num_days_sim = (date_end_sim-date_begin_sim).days


cases_obs = cov19.filter_one_country(confirmed_cases, country,
                                     date_data_begin, date_data_end)

print('Cases yesterday ({}): {} and '
      'day before yesterday: {}'.format(date_data_end.isoformat(), *cases_obs[:-3:-1]))

prior_date_mild_dist_begin =  datetime.datetime(2020,3,20)


change_points = [dict(pr_mean_date_begin_transient = prior_date_mild_dist_begin,
                      pr_sigma_date_begin_transient = 7,
                      pr_median_lambda = 0.2,
                      pr_sigma_lambda = 0.5),
              ]

traces = []
models = []

for num_change_points in range(2):
    model = cov19.SIR_with_change_points(new_cases_obs = np.diff(cases_obs),
                                        date_begin_simulation = date_begin_sim,
                                        num_days_sim = num_days_sim,
                                        diff_data_sim = diff_data_sim,
                                        N = 328e6,
                                        change_points_list=change_points
                                        )
    models.append(model)

    traces.append(pm.sample(model=model, init='advi', draws=1000))

Cases yesterday (2020-07-13T00:00:00): 3363056 and day before yesterday: 3304942
pr_beta_I_begin was set to default value 100
pr_median_lambda_0 was set to default value 0.4
pr_sigma_lambda_0 was set to default value 0.5
pr_median_mu was set to default value 0.125
pr_sigma_mu was set to default value 0.2
pr_median_delay was set to default value 8
pr_sigma_delay was set to default value 0.2
pr_beta_sigma_obs was set to default value 10
pr_median_transient_len of change point 0 was set to default value 3
pr_sigma_transient_len of change point 0 was set to default value 0.3


Auto-assigning NUTS sampler...
Initializing NUTS using advi...


In [ ]:
print('\n0 step model\n')
print(pm.loo(traces[0], model))

print('\n1 step model\n')
print(pm.loo(traces[1], model))


